attention机制

针对attention向量计算方式变体
大概分成这么几种：

Soft attention、global attention、动态attention

Hard attention

“半软半硬”的attention （local attention）

静态attention

强制前向attention

Soft attention、global attention、动态attention
这三个其实就是Soft attention，也就是我们上面讲过的那种最常见的attention，是在求注意力分配概率分布的时候，对于输入句子X中任意一个单词都给出个概率，是个概率分布，把attention变量（context vecor）用ct c_tc
https://blog.csdn.net/sun_xiao_kai/article/details/95873046

Hard attention
Soft是给每个单词都赋予一个单词match概率，那么如果不这样做，直接从输入句子里面找到某个特定的单词，然后把目标句子单词和这个单词对齐，而其它输入句子中的单词硬性地认为对齐概率为0，这就是Hard Attention Model的思想。

local attention （半软半硬attention）
Soft attention 每次对齐的时候都要考虑前面的encoder的所有hi，所以计算量会很大，因此一种朴素的思想是只考虑部分窗口内的encoder隐藏输出，其余部分为0，在窗口内使用softmax的方式转换为概率。这个local attention相反概念的是global attention，global attention其实就是softmax attention

静态attention
静态attention：对输出句子共用一个St的attention就够了，一般用在Bilstm的首位hidden state输出拼接起来作为st

self attention
思想：Self attention也叫做intra-attention在没有任何额外信息的情况下，我们仍然可以通过允许句子使用–self attention机制来处理自己，从句子中提取关注信息。

总的来说，attention的机制就是一个加权求和的机制，只要我们使用了加权求和，不管你是怎么花式加权，花式求和，只要你是根据了已有信息计算的隐藏状态的加权和求和，那么就是使用了attention，而所谓的self attention就是仅仅在句子内部做加权求和（区别与seq2seq里面的decoder对encoder的隐藏状态做的加权求和）。
self attention我个人认为作用范围更大一点，而key-value其实是对attention进行了一个更广泛的定义罢了，我们前面的attention都可以套上key-value attention，比如很多时候我们是把k和v都当成一样的算来，做self的时候还可能是quey=key=value。



In [8]:
import tensorflow as tf
def Attention(S): #S:batch_size,time_step,dim/hidden_size
     S_shape = S.shape.as_list()
     time_step = S_shape[1]
     hidden_size = S_shape[2]
     h_t = tf.Variable(tf.truncated_normal(shape=[hidden_size,1],stddev=0.5,dtype=tf.float32))
     W = tf.Variable(tf.truncated_normal(shape=[hidden_size, hidden_size], stddev=0.5, dtype=tf.float32))

     score = tf.matmul(  tf.matmul(tf.reshape(S,shape=[-1,hidden_size]),W)  ,h_t) # score: [batch_size*time_step,1]
     score = tf.reshape(score,shape=[-1,time_step,1]) # score: [batch_size,time_step,1]

     alpha_ts = tf.nn.softmax(score)## alpha:  [batch_size,time_step,1]
     c_t = tf.matmul(tf.transpose(S,[0,2,1]),alpha_ts)#[batch_size,dim,1]

     return tf.tanh(c_t)

S = tf.Variable(tf.truncated_normal(shape=[32,8,128],stddev=0.5,dtype=tf.float32))#[batch_size=32,time_step=8,dim=128]
res = Attention(S)
print(res)#

AttributeError: module 'tensorflow' has no attribute 'truncated_normal'

从概念上理解，即把Attention理解为从大量信息中有选择地筛选出少量重要信息并聚焦到这些重要信息上，忽略大多不重要的信息。这种思路仍然成立。

从数学角度讲，Attention用来形容是一个变量Query到一系列Key-Value对的映射，从公式上描述是根据Query和Key计算Values的加权求和的机制。

Attention计算主要分为三步:
第一步，是将Query和每个Key进行相似度计算得到权重，常用的相似度函数有点积，拼接，感知机等；
第二步，使用一个softmax函数对这些权重进行归一化；
第三 步，将获得的权重和相应的键值Value进行加权求和得到最后的attention。


什么是Self-attention？

对于self-attention来讲，Q(Query), K(Key), V(Value)三个矩阵均来自同一输入。

（1）因为如果是RNN或者LSTM，需要依次序序列计算，对于远距离的相互依赖的特征，要经过若干时间步步骤的信息累积才能将两者联系起来，而距离越远，有效捕获的可能性越小。
（2）Self Attention在计算过程中会直接将句子中任意两个单词的联系通过一个计算步骤直接联系起来，所以远距离依赖特征之间的距离被极大缩短，有利于有效地利用这些特征。
（3）引入Self Attention后会更容易捕获句子中长距离的相互依赖的特征

In [9]:
import torch
import torch.nn.functional as F

Q=torch.randn(16,28,300) #[batch_size*seq_len*emb]
K=torch.randn(16,28,300)
V=K

#获得注意力的权重
alpha_mat=torch.matmul(Q,K.transpose(1,2))
#归一化注意力权重，alpha把第一求和，softmax把第二位归一化
alpha=F.softmax(alpha_mat.sum(1,keepdim=True),dim=2)
#进行加权和
x=torch.matmul(alpha,V).squeeze(1)
print(x)

tensor([[ 0.1977, -1.1146,  2.2048,  ..., -0.4361,  1.3346,  0.7751],
        [ 2.0911, -1.6919,  1.2974,  ...,  1.5700, -0.7598, -0.2717],
        [-0.3789,  0.7023, -0.7611,  ..., -0.7543, -1.3017, -0.3186],
        ...,
        [-1.3382, -0.6928,  0.5357,  ..., -1.0540,  1.3277, -2.1235],
        [ 2.0068, -0.6286, -0.5871,  ..., -0.6938,  2.2665,  0.3095],
        [-1.3219,  0.2147, -0.0866,  ...,  1.5333, -0.0101,  1.3593]])


In [ ]:
import torch.nn as nn

def clones(module, N):
    """
    Produce N identical layers.
    """
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])


def attention(query, key, value, mask=None, dropout=None):
    """
    Compute 'Scaled Dot Product Attention'
    """
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn


class MultiHeadedAttention(nn.Module):
    """
    Implements 'Multi-Head Attention' proposed in the paper.
    """

    def __init__(self, h, d_model, dropout=0.1):
        """
        Take in model size and number of heads.
        """
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)

        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
                             for l, x in zip(self.linears, (query, key, value))]

        # 2) Apply attention on all the projected vectors in batch.
        x, self.attn = attention(query, key, value, mask=mask, dropout=self.dropout)

        # 3) "Concat" using a view and apply a final linear.
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)